In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
#ignore harmless warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Additional imports
from itertools import product
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tools.eval_measures import rmse
import matplotlib.pyplot as plt

In [3]:
# Define a function to iterate over different SARIMAX configurations
def find_best_sarimax(train, p_values, d_values, q_values, P_values, D_values, Q_values, m):
    best_aic = float("inf")
    best_order = None
    best_seasonal_order = None
    best_model = None
 

In [6]:
# Iterate over all combinations of (p, d, q) and (P, D, Q)
for p, d, q in product(p_values, d_values, q_values):
        for P, D, Q in product(P_values, D_values, Q_values):
            try:
                # Define the SARIMAX model
                model = SARIMAX(
                    train,
                    order=(p, d, q),
                    seasonal_order=(P, D, Q, m),
                    enforce_stationarity=False,
                    enforce_invertibility=False
                )

SyntaxError: incomplete input (3750880946.py, line 12)

In [ ]:
# Fit the model
                results = model.fit(disp=False)
                
                # Check if the current model has the best AIC
                if results.aic < best_aic:
                    best_aic = results.aic
                    best_order = (p, d, q)
                    best_seasonal_order = (P, D, Q, m)
                    best_model = results
            except Exception as e:
                # Handle any errors during fitting
                print(f"Error with order {(p, d, q)} and seasonal order {(P, D, Q, m)}: {e}")
                continue

    print(f"Best SARIMAX order: {best_order}, seasonal order: {best_seasonal_order}, AIC: {best_aic}")
    return best_model

In [ ]:
# Define ranges for the parameters
p = range(0, 3)     # p-values range from 0 to 2
d = range(0, 2)     # d-values range from 0 to 1
q = range(0, 3)     # q-values range from 0 to 2
P = range(0, 3)     # Seasonal P-values range from 0 to 2
D = range(0, 2)     # Seasonal D-values range from 0 to 1
Q = range(0, 3)     # Seasonal Q-values range from 0 to 2
m = 4               # Seasonal period is 4 (quarterly data)


In [ ]:
# Example training data
train = pd.DataFrame({'sick_leave_percentage': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]})


In [ ]:
# Assume test set for evaluation
test = pd.DataFrame({'sick_leave_percentage': [0.6, 0.7, 0.8, 0.9]})


In [ ]:
# Run the function to find the best model
best_model = find_best_sarimax(train['sick_leave_percentage'], p, d, q, P, D, Q, m)


In [ ]:
# Evaluate the best model on the test set
start = len(train)
end = len(train) + len(test) - 1
predictions = best_model.predict(start=start, end=end)


In [ ]:
# Calculate and print RMSE
rmse_value = rmse(test['sick_leave_percentage'], predictions)
print(f'RMSE: {rmse_value}')



In [ ]:
# Plot the results
ax = test['sick_leave_percentage'].plot(legend=True, figsize=(15,8), title='Sick Leave Percentage - Test vs Predictions')
predictions.plot(legend=True)
plt.show()